In [ ]:
import pandas as pd

In [19]:
df_hawkish = pd.read_csv('df_hawkish_dict_ver2.csv')
df_dovish = pd.read_csv('df_dovish_dict_ver2.csv')

In [1]:
# 사전 리스트
import pandas as pd
df_hawkish = pd.read_csv('version2_df_hawkish_dict.csv')
df_dovish = pd.read_csv('version2_df_dovish_dict.csv')
dic_hawkish = list(df_hawkish['ngram'])
dic_dovish = list(df_dovish['ngram'])

In [2]:
from tqdm.notebook import tqdm

In [3]:
import pandas as pd 
import numpy as np
import re

In [20]:
df_dic = pd.read_csv('ngramized_minutes_test.csv')#n_gram 만든 라벨링된 의사록
#-------------------------------필요없는 컬럼 제거하기=---------------------------------
df_dic = df_dic[['ID', 'date', 'sent', 'n_gram']]

In [21]:
#------------------------------n_gram split했을 때 이쁘게 뽑히게하기-----------------
for i in range(len(df_dic)):
    df_dic['n_gram'].iloc[i] = df_dic['n_gram'].iloc[i].replace("[","").replace("]","").replace("'","").replace(",","").split()

In [22]:
df_dic['hawkish'] = np.zeros(len(df_dic))
df_dic['dovish'] = np.zeros(len(df_dic))

In [25]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(df_dic))):
    for word in df_dic['n_gram'].iloc[i]:
        if word in dic_hawkish:
            df_dic['hawkish'].iloc[i] += 1

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(df_dic))):
    for word in df_dic['n_gram'].iloc[i]:
        if word in dic_dovish:
            df_dic['dovish'].iloc[i] += 1

In [27]:
df_dic[:5]

,ID,date,sent,n_gram,hawkish,dovish
0,2005년도 제10차,2005-05-12,['일부 위원은 내외금리차 역전에 따라 자금의 해외유출이 대규모로 발생될 가능성을 ...,"[내외/NNG, 금리갭/NNG, 역전/NNG, 자금/NNG, 해외/NNG, 유출/N...",4.0,0.0
1,2005년도 제10차,2005-05-12,"', '이에 대해 관련부서에서는 우리나라에서는 그러한 사례가 없었기 때문에 대만과 ...","[대해/VV, 그러/NNG, 사례/NNG, 없/VA, 사례/NNG, 살펴보/VV, ...",2.0,1.0
2,2005년도 제10차,2005-05-12,"', '동 위원은 자금의 해외유출에 대한 정책적 대응을 위해 내외금리나 환율에 대해...","[자금/NNG, 해외/NNG, 유출/NNG, 정책/NNG, 대응/NNG, 위하/VV...",2.0,0.0
3,2005년도 제10차,2005-05-12,"', '다른 일부 위원은 최근 금융기관의 외화대출이 늘어나고 있는 것과 관련하 여 ...","[금융기관/NNG, 외화대출/NNG, 늘/VV, 금융기관/NNG, 해외/NNG, 차...",0.0,2.0
4,2005년도 제10차,2005-05-12,"', '또 다른 일부 위원은 전년 동기와 비교할 때 금년 1～4월중 국내시장에 대 ...","[또/MAG, 비교/NNG, 하/VV, 때/NNG, 중/NNG, 시장/NNG, 대/...",16.0,1.0


In [28]:
df_dic['score_up'] = np.zeros(len(df_dic))

for i in range(len(df_dic)):
    score = (df_dic['hawkish'].iloc[i] - df_dic['dovish'].iloc[i]) / (df_dic['hawkish'].iloc[i] + df_dic['dovish'].iloc[i])
    if score > 0:
        df_dic['score_up'][i] = 1
    elif score < 0:
        df_dic['score_up'][i] = -1
    else:
        df_dic['score_up'][i] = 0
        
df_dic['score_down'] = np.zeros(len(df_dic))
for i in range(len(df_dic)):
    score = (df_dic['hawkish'].iloc[i] - df_dic['dovish'].iloc[i]) / (df_dic['hawkish'].iloc[i] + df_dic['dovish'].iloc[i])
    if score > 0:
        df_dic['score_down'][i] = 1
    elif score < 0:
        df_dic['score_down'][i] = 1
    else:
        df_dic['score_down'][i] = 0

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuf

In [29]:
polarity = df_dic.groupby(df_dic['date']).sum()
polarity[:5]

,hawkish,dovish,score_up,score_down
date,,,,
2005-05-12,552.0,75.0,100.0,126.0
2005-06-09,576.0,79.0,96.0,132.0
2005-07-07,606.0,108.0,93.0,127.0
2005-08-11,730.0,102.0,111.0,137.0
2005-09-08,698.0,123.0,105.0,135.0


In [30]:
#-----------------------------------0인값확인하기-------------------------------------
polarity[polarity['dovish'] == 0],polarity[polarity['hawkish'] == 0]

(Empty DataFrame
 Columns: [hawkish, dovish, score_up, score_down]
 Index: [],
 Empty DataFrame
 Columns: [hawkish, dovish, score_up, score_down]
 Index: [])

In [31]:
polarity['tone_doc'] = polarity['score_up'] / polarity['score_down']
polarity['tone_doc']

date
2005-05-12    0.793651
2005-06-09    0.727273
2005-07-07    0.732283
2005-08-11    0.810219
2005-09-08    0.777778
                ...   
2017-05-25    0.363636
2017-07-13    0.583333
2017-08-31    0.642857
2017-10-19    0.724138
2017-11-30    0.466667
Name: tone_doc, Length: 124, dtype: float64

In [32]:
#---------------------------------기준금리 합치기---------------------------------------

In [33]:
base_rate = pd.read_csv('df_baserate.csv')
base_rate[:5]

,날짜,기준금리
0,2005-05-01,3.25
1,2005-05-02,3.25
2,2005-05-03,3.25
3,2005-05-04,3.25
4,2005-05-05,3.25


In [34]:
df_new = polarity.merge(base_rate, left_on='date', right_on='날짜', how='left')
df_new[:5]

,hawkish,dovish,score_up,score_down,tone_doc,날짜,기준금리
0,552.0,75.0,100.0,126.0,0.793651,2005-05-12,3.25
1,576.0,79.0,96.0,132.0,0.727273,2005-06-09,3.25
2,606.0,108.0,93.0,127.0,0.732283,2005-07-07,3.25
3,730.0,102.0,111.0,137.0,0.810219,2005-08-11,3.25
4,698.0,123.0,105.0,135.0,0.777778,2005-09-08,3.25


In [35]:
df_new.corr()

,hawkish,dovish,score_up,score_down,tone_doc,기준금리
hawkish,1.000000,0.678638,0.922995,0.852850,0.491571,0.404186
dovish,0.678638,1.000000,0.650265,0.731797,-0.007209,0.250741
score_up,0.922995,0.650265,1.000000,0.969711,0.410623,0.410339
score_down,0.852850,0.731797,0.969711,1.000000,0.225047,0.355773
tone_doc,0.491571,-0.007209,0.410623,0.225047,1.000000,0.392979
기준금리,0.404186,0.250741,0.410339,0.355773,0.392979,1.000000
